In [3]:
#Importing the necessary libraries

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np

In [65]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
url = 'https://www.daraz.pk/'
driver.get(url)
search_bar = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_bar.send_keys('Mobile Phones' + Keys.RETURN)
next_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[3]/div/div/ul/li[9]/a')

In [66]:
driver.execute_script("window.scrollTo(0,500);")
time.sleep(2)
side_grid = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[2]/div/div[3]/div[2]/div/div[1]')
lines= side_grid.find_elements(By.CLASS_NAME, 'category-list__item--RCyN7')
for i in lines:
    if i.text == 'Mobiles':
        i.click()
        break

In [67]:
import time
import pandas as pd
import numpy as np
data=dict()
data["ID"]=[]
data["descriptions"] = []
data["prices"]= []
data["Brand"]=[]
data["rating"]=[]
data["free_shipping"]=[]
data["daraz_mall"]=[]
data["seller_rating"]=[]
data["ontime_rating"]=[]
count=1

temp = dict()
temp['ID']=[]
temp['Name'] = []
temp['Review text'] = []
#loop through pages 
for page in range(0,5):
    #find the ad grid and ad cards on the page
    ad_grid = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[2]')
    ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

#loop through each ad card
    for i in ad_cards:
        try:
            #extract information from the ad card
            data["descriptions"].append(i.find_element(By.CLASS_NAME, 'title--wFj93').text)
            data["prices"].append(i.find_element(By.CLASS_NAME, 'price--NVB62').text)
            data["free_shipping"].append(i.find_element(By.CLASS_NAME, 'location--eh0Ro').text)
            data['ID'].append(count)
            #check for Daraz Mall badge
            try:
                element = i.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[2]/div[39]/div/div/div[2]/div[1]/i')
                data["daraz_mall"].append(1)
            except NoSuchElementException:
                data["daraz_mall"].append(0)

            #click on the product link and switch to the new tab
            product_link_element = i.find_element(By.ID, 'id-a-link')
            product_link_element.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
            #in present tab
            time.sleep(2)
            try:
                data["Brand"].append(driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[5]/div/a[1]').text)
            except (NoSuchElementException,StaleElementReferenceException) as e :
                data["Brand"].append("not found")
            #extract information from the new tab
            percentage_grid = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]')
            boxes=percentage_grid.find_elements(By.CLASS_NAME, 'info-content')
            #extract seller rating and on-time rating
            try:
                data["seller_rating"].append(boxes[0].find_element(By.CLASS_NAME, 'rating-positive').text)
            except (NoSuchElementException,StaleElementReferenceException) as e :
                data["seller_rating"].append('0%')

            try:
                data["ontime_rating"].append(boxes[1].find_element(By.CLASS_NAME, 'seller-info-value').text)
            except (NoSuchElementException,StaleElementReferenceException) as e :
                data["ontime_rating"].append('0%')
            
            #scroll down to load more content
            driver.execute_script("window.scrollTo(0,1800);")
            
            time.sleep(5)
            #extract overall rating
            try:
                rating_box=driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[2]/div/div/div/div[1]/div[2]/div/div/div[1]')
                data["rating"].append(rating_box.find_element(By.CLASS_NAME, 'score-average').text)
            except (NoSuchElementException,StaleElementReferenceException) as e :
                data["rating"].append("not found")

            time.sleep(2)
            
            
            ####################reviews
            driver.execute_script("window.scrollTo(0, 500);")
            time.sleep(1)
            driver.execute_script("window.scrollTo(500, 800);")

            time.sleep(1)
            driver.execute_script("window.scrollTo(800, 1000);")
            time.sleep(2)

            #looping through review pages
            for page in range(0,3):
                try:
                    # Finding the container for reviews
                    reviews_container = driver.find_element(By.CLASS_NAME, 'review-content')
                except NoSuchElementException:
                    print('No Reviews')
                    break
                    #finding all individual reviews on the current page
                reviews = reviews_container.find_elements(By.CLASS_NAME,'review-item')

                #looping through each review on the current page
                for i in range(len(reviews)):
                    temp['ID'].append(count)
                    temp['Name'].append(reviews[i].find_element(By.CLASS_NAME,'user').text)
                    temp['Review text'].append(reviews[i].find_element(By.CLASS_NAME,'review-content-sl').text)

                try:
                    nxt_button = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[9]/button')
                    driver.execute_script("arguments[0].click();", nxt_button)
                    time.sleep(5)
                except NoSuchElementException:
                    print('No More Pages')
                    break
            #converting to dataframe
            reviews = pd.DataFrame(temp)
            reviews.to_csv('Reviews.csv')
            
            
            #close the new tab and switch back to the original tab
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except(NoSuchElementException,StaleElementReferenceException) as e :
            print(e)
            #convert to dataframe and save to csv after every page
        count+=1
        df = pd.DataFrame(data)
        df.to_csv('Products.csv')
        #click on next button to load page
    next_button.click()
    #add a delay to allow the new page to load
    time.sleep(5)
#     print(i)
#     print(page)



No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No Reviews
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No Reviews
No More Pages
No More Pages
No More Pages
No Reviews
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No Reviews
No More Pages
No More Pages
No More Pages
No More Pages
No Reviews
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No Reviews
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No More Pages
No Reviews
No More Pages

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00AF72A3+45731]
	(No symbol) [0x00A82D51]
	(No symbol) [0x0097880D]
	(No symbol) [0x0096A644]
	(No symbol) [0x0096A339]
	(No symbol) [0x0097A270]
	(No symbol) [0x009DB2F3]
	(No symbol) [0x009C7DD6]
	(No symbol) [0x009A31F6]
	(No symbol) [0x009A439D]
	GetHandleVerifier [0x00E00716+3229462]
	GetHandleVerifier [0x00E484C8+3523784]
	GetHandleVerifier [0x00E4214C+3498316]
	GetHandleVerifier [0x00B81680+611968]
	(No symbol) [0x00A8CCCC]
	(No symbol) [0x00A88DF8]
	(No symbol) [0x00A88F1D]
	(No symbol) [0x00A7B2C7]
	BaseThreadInitThunk [0x774C6839+25]
	RtlGetFullPathName_UEx [0x77AF906F+1215]
	RtlGetFullPathName_UEx [0x77AF903D+1165]


In [6]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def tokenize_input(user_input):
    tokens = word_tokenize(user_input)#.lower())
    return [word for word in tokens]

query=input("Enter mobile that u want: ")
tokenized_query=tokenize_input(query)

data = pd.read_csv('Products.csv')
df = pd.DataFrame(data)
matching_indexes=[]
for word in tokenized_query:
    #for Brand
    for index, brand in enumerate(df['Brand']):
        if word in brand:
            matching_indexes.append(index)
    
for i in matching_indexes:
    print(df['prices'][i])
    
        
    

Enter mobile that u want: Samsung phone
Rs. 34,999
Rs. 54,999
Rs. 72,999
Rs. 42,999
Rs. 51,999
Rs. 29,999
Rs. 128,999
Rs. 39,999
Rs. 34,999
Rs. 54,999
Rs. 72,999
Rs. 42,999
Rs. 51,999
Rs. 29,999
Rs. 128,999
Rs. 39,999
Rs. 29,999
Rs. 134,999
Rs. 28,699
Rs. 139,999
Rs. 23,499
Rs. 53,999
Rs. 38,999
Rs. 43,499
Rs. 135,999
Rs. 134,999


In [69]:
#programming the chatbot

#tokenizing names and then matching from tokens of user's input sentence

'''

import necessary libraries

'''
# import pandas library which is required for functionalities with your dataframe

import pandas as pd

'''
import nltk library (Natural Language Toolkit)

'''
import nltk

'''

from nltk library import word tokenize and sentence tokenizing features

'''
from nltk.tokenize import word_tokenize,sent_tokenize

'''

download punkt module from nltk, The punkt module is responsible for tokenization

'''
nltk.download('punkt')

df=pd.read_csv('Products.csv')
'''

now we will tokenize all words of column name of dataframe containing all extracted information from scraping from daraz

'''
#tokenize all words in the 'name' column and create a set of unique tokens

'''

Tokenize all words in the 'name' column, convert to lowercase, and create a set of unique tokens
'''
name_tokens = set(word_tokenize(' '.join(df['name']).lower()))

'''

function to tokenize input sentence entered by user

'''
def tokens(user_sentence):
    
    #tokenize user's sentence by .lower() function and then yield it
    return word_tokenize(user_sentence.lower())

'''

not extract mobile information based on words that match in name column and user's input sentence

'''

#function to retrieve mobile information based on user input

#this function takes 3 parameters user_ sentence to convert it to tokens and match it to tokens of column name tokenized above and dataframe df

def mobile_info(user_sentence, name_tokens, df):
    
    #word tokenize user input
    words = tokens(user_sentence)
    '''
    
    now find words which are common in tokens of user sentence and tokens of name columns that stores names of mobile
    
    '''
    #find common tokens between user input and mobile names

    final = set(words) & name_tokens
    
    #if words doesnt match then display a message to user that products is not found
    
    if not final:
        
        #display message that this product is not found
        
        return "Sorry, I couldn't identify any common words with mobile names in your input"

    #filter DataFrame based on common tokens
    
    matched= df[df['name'].str.lower().apply(lambda x: any(token in x for token in final))]

    #if no information is gathered then display the message that no phones found
    
    if matched .empty:
        # display that no mobiles match the words in user's sentence
        return "Sorry, I couldn't find any mobile phones matching the words in your input."

    #display information about the matching products
    result = ""
    
    #iterate through the matched rows in the DataFrame
    for index, row in matched.iterrows():
        
        #concatenate information about the product to the result string
        result += "Product: {}\nPrice: {}\nBrand: {}\nReviews: {}\n\n".format(row['name'], row['price'], row['brand'], row['reviews'])
        
    #return the final result string
    
    return result

#Prompt user to enter sentence

user_sentence = input("Enter a sentence about the phones you want: ")
#get information based on user input and print the result
output = mobile_info(user_sentence, name_tokens, df)

#print output based on users' demand
print(output)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


FileNotFoundError: [Errno 2] No such file or directory: 'Products.csv'

In [36]:
import time
import pandas as pd
import numpy as np
data=dict()
data["descriptions"] = []
data["prices"]= []
# data["ID"]=[]
data["Brand"]=[]


temp = dict()
temp['ID'] = []
temp['Review text'] = []
temp['score']=[]
for page in range(0,5):
    #find the ad grid and ad cards on the page
    ad_grid = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[2]')
    ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

#loop through each ad card
    for i in ad_cards:
        try:
            #extract information from the ad card
            data["descriptions"].append(i.find_element(By.CLASS_NAME, 'title--wFj93').text)
            data["prices"].append(i.find_element(By.CLASS_NAME, 'price--NVB62').text)

            product_link_element = i.find_element(By.ID, 'id-a-link')
            product_link_element.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
            #in present tab
            time.sleep(2)
           
            try:
                data["Brand"].append(driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[5]/div/a[1]').text)
            except (NoSuchElementException,StaleElementReferenceException) as e :
                data["Brand"].append("not found")
            
            
            
            #scrolling down the page to load reviews
            driver.execute_script("window.scrollTo(0, 500);")
            time.sleep(1)
            driver.execute_script("window.scrollTo(500, 800);")
            time.sleep(1)
            driver.execute_script("window.scrollTo(800, 1000);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(1000, 1300);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(1300, 1600);")
            time.sleep(1)

            #looping through review pages
            for page in range(0,3):
                try:
                    # Finding the container for reviews
                    reviews_container = driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[1]')
                except NoSuchElementException:
                    print('No Reviews')
                    break
                    #finding all individual reviews on the current page
                reviews = reviews_container.find_elements(By.CLASS_NAME,'item')

                #looping through each review on the current page
                for i in range(len(reviews)):
                    #checking if the review text is not empty
                    if reviews[i].find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({i+1}) > div.item-content > div.content").text != '':
                        #extracting and appending reviewer name and review text
                        temp['Name'].append(reviews[i].find_element(By.CLASS_NAME,'middle').find_element(By.TAG_NAME,'span').text[3:])
                        temp['Review text'].append(reviews[i].find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({i+1}) > div.item-content > div.content").text)
                        temp['sentiment'].append('x')
                try:
                    #clicking the next button if it exists and not on the third page
                    if(page != 2):
                        nxt_button = driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[2]/div/button[2]')
                        driver.execute_script("arguments[0].click();", nxt_button)
                        time.sleep(5)
                except NoSuchElementException:
                    print('No More Pages')
                    break
            #converting to dataframe
            reviews = pd.DataFrame(temp)

            
            
            
            time.sleep(2)
            #close the new tab and switch back to the original tab
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except(NoSuchElementException,StaleElementReferenceException) as e :
            print(e)
            #convert to dataframe and save to csv after every page
        df = pd.DataFrame(data)
#         df.to_csv('Q3_data.csv')
        #click on next button to load page
    next_button.click()
    #add a delay to allow the new page to load
    time.sleep(5)
#     print(i)
#     print(page)



No More Pages
No More Pages
No More Pages


AttributeError: 'NoneType' object has no attribute 'text'

In [68]:
pip install nltk


In [38]:
reviews

,ID,Review text,score


In [47]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
url='https://www.daraz.pk/products/12-8gb-128gb-5000mah-33-i421379488-s1989920618.html?spm=a2a0e.searchlist.sku.1.29313de2gohjjQ&search=1'
driver.get(url)

In [51]:
import time
import pandas as pd



temp = dict()
temp['Name'] = []
temp['Review text'] = []
temp['sentiment']=[]
#scrolling down the page to load reviews
driver.execute_script("window.scrollTo(0, 500);")
time.sleep(1)
driver.execute_script("window.scrollTo(500, 800);")

time.sleep(1)
driver.execute_script("window.scrollTo(800, 1000);")
time.sleep(2)

#looping through review pages
for page in range(0,3):
    try:
        # Finding the container for reviews
        reviews_container = driver.find_element(By.CLASS_NAME, 'review-content')
    except NoSuchElementException:
        print('No Reviews')
        break
        #finding all individual reviews on the current page
    reviews = reviews_container.find_elements(By.CLASS_NAME,'review-item')

    #looping through each review on the current page
    for i in range(len(reviews)):
        temp['Name'].append(reviews[i].find_element(By.CLASS_NAME,'user').text)
        temp['Review text'].append(reviews[i].find_element(By.CLASS_NAME,'review-content-sl').text)

    try:
        nxt_button = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[9]/button')
        driver.execute_script("arguments[0].click();", nxt_button)
        time.sleep(5)
    except NoSuchElementException:
        print('No More Pages')
        break
#converting to dataframe
reviews = pd.DataFrame(temp)

In [52]:
reviews


,Name,Review text,sentiment
0,Abdul G.,Assalam u Alaikum Received before due date ori...,x
1,S***n,The seller was very cooperative and delivered ...,x
2,*******686,Same as described.. Amazing Packing and receiv...,x
3,S***n,The seller was very cooperative and delivered ...,x
4,3***9,Masha Allah Jo select kia tha wohi same receiv...,x
5,Abeel J.,Received within two days same as shown in adve...,x
6,N***.,Absolutely amazing product and 5 star to all s...,x
7,*******686,Same as described.. Amazing Packing and receiv...,x
8,sidra 0.,jo mng waya tha wohi hai thnkz daraz,x
